In [1]:
import pandas as pd
import numpy as np
import json
from collections import Counter

## data load

In [2]:
new = pd.read_csv('new_sample_song_tag.csv')
print(new.shape)
new.head() 

(1117, 13)


,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tags
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,헬스 스포츠 피트니스 강렬한 운동 EDM 트랜스
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,스카펑크 1990년대 록 펑크록 스트레스 펑크 신나는락
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,힙합 랩스트레스 기분전환
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,휴식 힐링 스트레스팝팝
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,스트레스 밤 새벽


## 태그 정리
- 리스트 형태로 되어 있는 tag를 문자열로 바꿈

In [3]:
new['tags'] = [new['tags'][i].split() for i in range(len(new))]
new['tags'] = [', '.join(new['tags'][i]) for i in range(len(new))]
new.head()

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tags
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,"헬스, 스포츠, 피트니스, 강렬한, 운동, EDM, 트랜스"
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,"스카펑크, 1990년대, 록, 펑크록, 스트레스, 펑크, 신나는락"
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,"힙합, 랩스트레스, 기분전환"
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,"휴식, 힐링, 스트레스팝팝"
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,"스트레스, 밤, 새벽"


## 사용하기로 한 tag만 남기고 정리
- 감정이랑 매칭시킨 tag 데이터 가져와서 사용

In [4]:
tag_p = pd.read_excel('./data/tag_8emotion.xlsx', sheet_name = 'Sheet1')
tag_p.head()

,emotion,tag
0,무서운,밤
1,긴장되는,스트레스
2,긴장되는,고백
3,긴장되는,출근
4,긴장되는,섹시


In [5]:
# 태그 리스트 생성
t_all = list(tag_p['tag'])

# 감정 리스트 생성
emotions = list(tag_p['emotion'].unique())

In [6]:
df = new.copy() 
df.head()

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tags
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,"헬스, 스포츠, 피트니스, 강렬한, 운동, EDM, 트랜스"
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,"스카펑크, 1990년대, 록, 펑크록, 스트레스, 펑크, 신나는락"
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,"힙합, 랩스트레스, 기분전환"
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,"휴식, 힐링, 스트레스팝팝"
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,"스트레스, 밤, 새벽"


In [7]:
# 우리가 사용하기로 한 tag만 남겨둔 리스트
tag_in = [[df['tags'][i].split(', ')[j] for j in range(len(df['tags'][i].split(', '))) if df['tags'][i].split(', ')[j] in t_all] for i in range(len(df))] 

# 전체 tag 대신 tag_in을 column으로 넣고 우리가 사용하기로 한 tag만 사용
df['tag_in'] = pd.Series(tag_in)

df.head()

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tags,tag_in
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,"헬스, 스포츠, 피트니스, 강렬한, 운동, EDM, 트랜스","[헬스, 스포츠, 강렬한, 운동]"
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,"스카펑크, 1990년대, 록, 펑크록, 스트레스, 펑크, 신나는락",[스트레스]
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,"힙합, 랩스트레스, 기분전환",[기분전환]
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,"휴식, 힐링, 스트레스팝팝","[휴식, 힐링]"
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,"스트레스, 밤, 새벽","[스트레스, 밤, 새벽]"


## tag 없는 노래 삭제

In [8]:
df['tag_in'] = pd.Series([', '.join(df['tag_in'][i]) for i in range(len(df))])
df = df[df['tag_in'] != '']

df.reset_index(inplace = True) 
df.drop(['level_0', 'tags'], axis = 1, inplace = True) 

df.head()

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tag_in
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,"헬스, 스포츠, 강렬한, 운동"
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,스트레스
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,기분전환
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,"휴식, 힐링"
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,"스트레스, 밤, 새벽"


In [9]:
df.shape

(1078, 13)

In [10]:
1117-1078

39

## 사용하기로 한 장르만 남기고 정리
- 전에 회의에서 정한대로 

In [11]:
gnrs = pd.read_excel('./data/gnr_code.xlsx')
gnrs.head() 

,장르 대분류,대분류 설명,장르 소분류,소분류 설명
0,GN0100,발라드,GN0101,세부장르전체
1,GN0100,발라드,GN0102,'80
2,GN0100,발라드,GN0103,'90
3,GN0100,발라드,GN0104,'00
4,GN0100,발라드,GN0105,'10-


In [12]:
gnrs.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   장르 대분류  225 non-null    object
 1   대분류 설명  225 non-null    object
 2   장르 소분류  224 non-null    object
 3   소분류 설명  224 non-null    object
dtypes: object(4)
memory usage: 7.2+ KB


In [13]:
# gnr 리스트 생성
gnr_all = list(gnrs['장르 대분류'].unique())

# gnr 소분류 리스트 생성
gnr_dtl_all = list(gnrs['장르 소분류'].unique())

In [14]:
df

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tag_in
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,"헬스, 스포츠, 강렬한, 운동"
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,스트레스
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,기분전환
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,"휴식, 힐링"
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,"스트레스, 밤, 새벽"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,"['GN1014', 'GN0901', 'GN0902', 'GN1001']",20180921,The Click (Deluxe Edition),10206107,[770539],Sober Up (feat. Rivers Cuomo),"['GN0900', 'GN1000']",['AJR'],428602,"GN0900, GN1000","GN1014, GN0901, GN0902, GN1001",428602,"기분좋은, 따스한, 힘들때, 일상, 감미로운"
1074,['GN0901'],20170713,도심을 거닐며 듣는 팝 음악 (City Pop Music),10078776,[750470],I Wish,['GN0900'],['Stan Tristan'],652060,GN0900,GN0901,652060,"퇴근, 분위기, 집, 기분전환"
1075,"['GN1401', 'GN1404']",20161209,The Complete Capitol Singles: 1957-1966,10021644,"[28022, 1272181]",I`ve Got A Tiger By The Tail (Mono Single Ver.),['GN1400'],"['Buck Owens', 'The Buckeroos']",178922,GN1400,"GN1401, GN1404",178922,편안한
1076,"['GN1601', 'GN1606']",20140825,The Mozart Album,2286844,[346065],Mozart : Piano Sonata No.8 In A Minor K.310 - ...,['GN1600'],['Lang Lang'],7658,GN1600,"GN1601, GN1606",7658,"퇴근, 힐링"


## 사용하기로 장르 __대분류__만 남기고 정리

In [36]:
# 대분류 있는 것만 남김
ok = []
for i in range(len(df)):
    try:
        ok.append(len(df['대분류str'][i]) > 4)
    except:
        ok.append(False) 

In [39]:
len(ok) == len(df) 

True

In [41]:
df = df[ok]
df

,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tag_in
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,"헬스, 스포츠, 강렬한, 운동"
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,스트레스
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,기분전환
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,"휴식, 힐링"
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,"스트레스, 밤, 새벽"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,"['GN1014', 'GN0901', 'GN0902', 'GN1001']",20180921,The Click (Deluxe Edition),10206107,[770539],Sober Up (feat. Rivers Cuomo),"['GN0900', 'GN1000']",['AJR'],428602,"GN0900, GN1000","GN1014, GN0901, GN0902, GN1001",428602,"기분좋은, 따스한, 힘들때, 일상, 감미로운"
1074,['GN0901'],20170713,도심을 거닐며 듣는 팝 음악 (City Pop Music),10078776,[750470],I Wish,['GN0900'],['Stan Tristan'],652060,GN0900,GN0901,652060,"퇴근, 분위기, 집, 기분전환"
1075,"['GN1401', 'GN1404']",20161209,The Complete Capitol Singles: 1957-1966,10021644,"[28022, 1272181]",I`ve Got A Tiger By The Tail (Mono Single Ver.),['GN1400'],"['Buck Owens', 'The Buckeroos']",178922,GN1400,"GN1401, GN1404",178922,편안한
1076,"['GN1601', 'GN1606']",20140825,The Mozart Album,2286844,[346065],Mozart : Piano Sonata No.8 In A Minor K.310 - ...,['GN1600'],['Lang Lang'],7658,GN1600,"GN1601, GN1606",7658,"퇴근, 힐링"


In [55]:
df.reset_index(drop = True, inplace = True) 
df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tag_in,gnr_in
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,"헬스, 스포츠, 강렬한, 운동",[GN2700]
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,스트레스,[GN1000]
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,기분전환,[GN1200]
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,"휴식, 힐링",[GN0900]
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,"스트레스, 밤, 새벽",[GN1000]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,"['GN1014', 'GN0901', 'GN0902', 'GN1001']",20180921,The Click (Deluxe Edition),10206107,[770539],Sober Up (feat. Rivers Cuomo),"['GN0900', 'GN1000']",['AJR'],428602,"GN0900, GN1000","GN1014, GN0901, GN0902, GN1001",428602,"기분좋은, 따스한, 힘들때, 일상, 감미로운","[GN0900, GN1000]"
1034,['GN0901'],20170713,도심을 거닐며 듣는 팝 음악 (City Pop Music),10078776,[750470],I Wish,['GN0900'],['Stan Tristan'],652060,GN0900,GN0901,652060,"퇴근, 분위기, 집, 기분전환",[GN0900]
1035,"['GN1401', 'GN1404']",20161209,The Complete Capitol Singles: 1957-1966,10021644,"[28022, 1272181]",I`ve Got A Tiger By The Tail (Mono Single Ver.),['GN1400'],"['Buck Owens', 'The Buckeroos']",178922,GN1400,"GN1401, GN1404",178922,편안한,[GN1400]
1036,"['GN1601', 'GN1606']",20140825,The Mozart Album,2286844,[346065],Mozart : Piano Sonata No.8 In A Minor K.310 - ...,['GN1600'],['Lang Lang'],7658,GN1600,"GN1601, GN1606",7658,"퇴근, 힐링",[GN1600]


In [48]:
### 장르 대분류 정리
# 우리가 사용하기로 한 장르만 남겨둔 리스트
gnr_in = [[df['대분류str'][i].split(', ')[j] for j in range(len(df['대분류str'][i].split(', '))) if df['대분류str'][i].split(', ')[j] in gnr_all] for i in range(len(df))] 
# 대분류str 대신 gnr_in을 column으로 넣음
df['gnr_in'] = pd.Series(gnr_in)

<ipython-input-48-0c9ba7f80220>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gnr_in'] = pd.Series(gnr_in)


In [49]:
df

,level_0,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tag_in,gnr_in
0,0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,"헬스, 스포츠, 강렬한, 운동",[GN2700]
1,1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,스트레스,[GN1000]
2,2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,기분전환,[GN1200]
3,3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,"휴식, 힐링",[GN0900]
4,4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,"스트레스, 밤, 새벽",[GN1000]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,1073,"['GN1014', 'GN0901', 'GN0902', 'GN1001']",20180921,The Click (Deluxe Edition),10206107,[770539],Sober Up (feat. Rivers Cuomo),"['GN0900', 'GN1000']",['AJR'],428602,"GN0900, GN1000","GN1014, GN0901, GN0902, GN1001",428602,"기분좋은, 따스한, 힘들때, 일상, 감미로운","[GN0900, GN1000]"
1034,1074,['GN0901'],20170713,도심을 거닐며 듣는 팝 음악 (City Pop Music),10078776,[750470],I Wish,['GN0900'],['Stan Tristan'],652060,GN0900,GN0901,652060,"퇴근, 분위기, 집, 기분전환",[GN0900]
1035,1075,"['GN1401', 'GN1404']",20161209,The Complete Capitol Singles: 1957-1966,10021644,"[28022, 1272181]",I`ve Got A Tiger By The Tail (Mono Single Ver.),['GN1400'],"['Buck Owens', 'The Buckeroos']",178922,GN1400,"GN1401, GN1404",178922,편안한,[GN1400]
1036,1076,"['GN1601', 'GN1606']",20140825,The Mozart Album,2286844,[346065],Mozart : Piano Sonata No.8 In A Minor K.310 - ...,['GN1600'],['Lang Lang'],7658,GN1600,"GN1601, GN1606",7658,"퇴근, 힐링",[GN1600]


## 사용하기로 한 장르 __소분류__만 남기고 정리

In [57]:
# 소분류 있는 것만 남김
ok = []
for i in range(len(df)):
    try:
        ok.append(len(df['소분류str'][i]) > 4)
    except:
        ok.append(False) 
len(ok) == len(df)

True

In [59]:
df = df[ok]
print(df.shape)
df.head() 

(1028, 14)


,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tag_in,gnr_in
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,"헬스, 스포츠, 강렬한, 운동",[GN2700]
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,스트레스,[GN1000]
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,기분전환,[GN1200]
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,"휴식, 힐링",[GN0900]
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,"스트레스, 밤, 새벽",[GN1000]


In [60]:
df.reset_index(drop = True, inplace = True)

In [61]:
### 장르 소분류 정리
# 우리가 사용하기로 한 장르 소분류만 남겨둔 리스트
gnr_dtl_in = [[df['소분류str'][i].split(', ')[j] for j in range(len(df['소분류str'][i].split(', '))) if df['소분류str'][i].split(', ')[j] in gnr_dtl_all] for i in range(len(df))] 
# 소분류str 대신 gnr_dtl_in을 column으로 넣음
df['gnr_dtl_in'] = pd.Series(gnr_dtl_in)

df.head() 

<ipython-input-61-2478a05915cb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gnr_dtl_in'] = pd.Series(gnr_dtl_in)


,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,대분류str,소분류str,index,tag_in,gnr_in,gnr_dtl_in
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,GN2700,"GN2701, GN2703",278446,"헬스, 스포츠, 강렬한, 운동",[GN2700],"[GN2701, GN2703]"
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,GN1000,"GN1013, GN1001",597527,스트레스,[GN1000],"[GN1013, GN1001]"
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,GN1200,GN1201,358922,기분전환,[GN1200],[GN1201]
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,GN0900,GN0901,669719,"휴식, 힐링",[GN0900],[GN0901]
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,GN1000,GN1001,520113,"스트레스, 밤, 새벽",[GN1000],[GN1001]


## 남은 장르 없는 노래 삭제

In [62]:
df['gnr_in'] = pd.Series([', '.join(df['gnr_in'][i]) for i in range(len(df))])
df = df[df['gnr_in'] != '']

df['gnr_dtl_in'] = pd.Series([', '.join(df['gnr_dtl_in'][i]) for i in range(len(df))])
df = df[df['gnr_dtl_in'] != '']

df.reset_index(inplace = True) 
df.drop(['level_0', '대분류str', '소분류str'], axis = 1, inplace = True) 

df.head()

<ipython-input-62-df7ea583cb7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gnr_in'] = pd.Series([', '.join(df['gnr_in'][i]) for i in range(len(df))])


,곡 세부 장르 리스트,발매일,앨범 명,앨범 ID,아티스트 ID 리스트,곡 제목,곡 장르 리스트,아티스트 리스트,곡 ID,index,tag_in,gnr_in,gnr_dtl_in
0,"['GN2701', 'GN2703']",20190226,Stand By You,10254717,"[720972, 41521, 928015]",Stand By You,['GN2700'],"['Solis & Sean Truby', 'Ultimate', 'Katherine ...",278446,278446,"헬스, 스포츠, 강렬한, 운동",GN2700,"GN2701, GN2703"
1,"['GN1013', 'GN1001']",20080318,Hits,374361,[106037],Hit Or Miss,['GN1000'],['New Found Glory'],597527,597527,스트레스,GN1000,"GN1013, GN1001"
2,['GN1201'],20140120,Hip-Hop - The Old Skool Collection,2229601,[164257],The Creator (Remastered LP Ver.),['GN1200'],['Pete Rock & C.L. Smooth'],358922,358922,기분전환,GN1200,GN1201
3,['GN0901'],20151211,North Star (Bloody Christmas),2655644,[716506],North Star (Bloody Christmas),['GN0900'],['Elliphant'],669719,669719,"휴식, 힐링",GN0900,GN0901
4,['GN1001'],20150807,Put Your Arms Around Me (Remixes),2333915,[100354],Put Your Arms Around Me (Spooky Soul Mix),['GN1000'],['Texas'],520113,520113,"스트레스, 밤, 새벽",GN1000,GN1001


In [63]:
df.shape

(1028, 13)

In [65]:
1117 - 1028

89

In [64]:
df.to_csv('sampling_song_id.csv', index = False)